<a href="https://colab.research.google.com/github/adundulis/stable-diffusion/blob/main/lora_stable_diffusion_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA Stable Diffusion

In [ ]:
#@markdown # Setup LoRA, Dependencies and Mount Google Drive

from IPython.utils import capture
import sys
import os
import time

SESSION_NAME = "char" #@param{type: 'string'}

if SESSION_NAME == "":
  raise SystemExit('SESSION_NAME should not be empty.')

print('Mounting Google Drive...')
with capture.capture_output() as cap:
  from google.colab import drive
  drive.mount('/content/gdrive')

print('Installing LoRA and Dependencies...')
with capture.capture_output() as cap:
  !git clone https://github.com/cloneofsimo/lora.git && sed -i 's/functools.cache/functools.lru_cache(maxsize=None)/g' /content/lora/lora_diffusion/xformers_utils.py && pip install /content/lora
  !pip install accelerate bitsandbytes

print('Intialising Session Workspace...')
WORKSPACE='/content/gdrive/MyDrive/lora-stable-difussion'
SESSION_DIR = WORKSPACE + '/sessions/' + SESSION_NAME
IMAGES_DIR = SESSION_DIR + '/images'
CAPTIONS_DIR = SESSION_DIR + '/captions'
MODELS_DIR = SESSION_DIR + '/models'

with capture.capture_output() as cap:
  %cd /content
  if not os.path.exists(str(SESSION_DIR)):
    %mkdir -p $SESSION_DIR
    time.sleep(1)
  if not os.path.exists(str(IMAGES_DIR)):
    %mkdir -p $IMAGES_DIR
    time.sleep(1)
  if not os.path.exists(str(CAPTIONS_DIR)):
    %mkdir -p $CAPTIONS_DIR
    time.sleep(1)
  if not os.path.exists(str(MODELS_DIR)):
    %mkdir -p $MODELS_DIR
    time.sleep(1)

print('\033[92m' + 'Initialisation Successful' + '\033[0m')

In [7]:
#@markdown # Upload Images and Captions

import shutil
from google.colab import files
from IPython.display import display, clear_output

UPLOAD_IMAGES = True #@param{type: 'boolean'}
UPLOAD_CAPTIONS = True #@param{type: 'boolean'}
REMOVE_EXISTING_IMAGES = True #@param{type: 'boolean'}
REMOVE_EXISTING_CAPTIONS = True #@param{type: 'boolean'}

if REMOVE_EXISTING_IMAGES:
  if os.path.exists(str(IMAGES_DIR)):
    !rm -r "$IMAGES_DIR"
if not os.path.exists(str(IMAGES_DIR)):
  %mkdir -p "$IMAGES_DIR"

if REMOVE_EXISTING_CAPTIONS:
  if os.path.exists(str(CAPTIONS_DIR)):
    !rm -r "$CAPTIONS_DIR"
if not os.path.exists(str(CAPTIONS_DIR)):
  %mkdir -p "$CAPTIONS_DIR"

print('Upload Images')
uploaded_images = files.upload()
for filename in uploaded_images.keys():
  shutil.move(filename, IMAGES_DIR)

clear_output(wait=True)

print('Upload Captions')
uploaded_captions = files.upload()
for filename in uploaded_captions.keys():
  shutil.move(filename, CAPTIONS_DIR)

clear_output(wait=True)

print('\033[92m' + 'Upload Completed' + '\033[0m')

Upload Completed


In [ ]:
STEPS = 300 #@param {type:"slider", min:0, max:10000, step:10}
BATCH_SIZE = 1 #@param {type:"slider", min:0, max:128, step:1}
FP_16 = True #@param {type:"boolean"}

#@markdown ----
#@markdown UNET PARAMS
LEARNING_RATE = 3e-4 #@param {type:"number"}

#@markdown ----
TRAIN_TEXT_ENCODER = True #@param {type:"boolean"}
#@markdown TEXT ENCODER PARAMS
LEARNING_RATE_TEXT_ENCODER = e-5 #@param {type:"number"}

NEW_LEARNING_RATE = LEARNING_RATE / BATCH_SIZE
NEW_LEARNING_RATE_TEXT_ENCODER = LEARNING_RATE_TEXT_ENCODER / BATCH_SIZE

if FP_16:
  fp_16_arg = "fp16"
else:
  fp_16_arg = "no"

if TRAIN_TEXT_ENCODER:
  command = (f'accelerate launch lora/training_scripts/train_lora_dreambooth.py '
             f'--pretrained_model_name_or_path="{PRETRAINED_MODEL}" '
             f'--instance_data_dir="{INSTANCE_DIR}" '
             f'--output_dir="{OUTPUT_DIR}" '
             f'--instance_prompt="{PROMPT}" '
             f'--resolution=512 '
             f'--use_8bit_adam '
             f'--mixed_precision="{fp_16_arg}" '
             f'--train_batch_size=1 '
             f'--gradient_accumulation_steps=1 '
             f'--learning_rate={NEW_LEARNING_RATE} '
             f'--lr_scheduler="constant" '
             f'--lr_warmup_steps=0 '
             f'--max_train_steps={STEPS} '
             f'--train_text_encoder '
             f'--lora_rank=16 '
             f'--learning_rate_text={NEW_LEARNING_RATE_TEXT_ENCODER}')
else:
  command = (f'accelerate launch lora/training_scripts/train_lora_dreambooth.py '
             f'--pretrained_model_name_or_path="{PRETRAINED_MODEL}" '
             f'--instance_data_dir="{INSTANCE_DIR}" '
             f'--output_dir="{OUTPUT_DIR}" '
             f'--instance_prompt="{PROMPT}" '
             f'--resolution=512 '
             f'--use_8bit_adam '
             f'--mixed_precision="{fp_16_arg}" '
             f'--train_batch_size=1 '
             f'--gradient_accumulation_steps=1 '
             f'--learning_rate={NEW_LEARNING_RATE} '
             f'--lr_scheduler="constant" '
             f'--lr_warmup_steps=0 '
             f'--lora_rank=16 '
             f'--max_train_steps={STEPS} '
             f'--learning_rate_text={NEW_LEARNING_RATE_TEXT_ENCODER}')
!rm -rf $INSTANCE_DIR/.ipynb_checkpoints
!{command}